<a href="https://colab.research.google.com/github/nmalby/APS/blob/rewrite/APS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#APS Explorer

In [1]:
import math

##Parameters

In [ ]:
'''
Parameters
  Configuration
    Shell
      Loader      - Belt, 1, 2, 3, 4, 6, 7, 8             - Multiple            CheckBox
      Modules
        Head      - AP, Heavy, AP, Thump, Chem, Special   - Multiple            Checkbox
        Fuse      - None, Single, Double                  - Single              Radio
        Base      - None, BB, Trace, Cavitate             - Single              Radio
    Gun
      Gauge       - 18-500                                - Single              Field/Slider
      Barrels     - 1-6                                   - Single              Radio
      Evacuator   - Y/N                                   - Single              Radio
      Ejector     - Y/N                                   - Single              Radio
      Rail        - Y/N                                   - Single              Radio
      Rail Limit  - 1000-20000                            - Single              Field/Slider

    Engine
      PPM         - 200-800                               - Single              Field/Slider
      PPV         - 10-200                                - Single              Field/Slider
      PPB         - 10-200                                - Single              Field/Slider

  Requirements
    AP            - 0-60                                  - Single              Field/Slider
    Velocity      - 500-2000                              - Multiple (min-max)  Slider
    (Pen Req)     -                                       - Single              Field

  Goal Weights
    KD            - 0-1                                   - Single              Field/Slider
    CD            - 0-1                                   - Single              Field/Slider
    ROF           - 0-1                                   - Single              Field/Slider
    (Uptime)      - 0-1 (Belt)                            - Single              Field/Slider
    Volume        - 0-1                                   - Single              Field/Slider
    Cost          - 0-1 (Build + 10min)                   - Single              Field/Slider
'''

In [ ]:
constraints = {
    "Configuration": {
        "Shell": {

            "Modules": {
                "Head": ["AP", "Heavy", "Sabot", "Thump", "Chem", "Special"],
                "Fuse": ["None", "Single", "Double"],
                "Base": ["None", "BB", "Trace", "Cavitate"]
            }
        },
        "Gun": {
            "Loader": ["Belt", 1, 2, 3, 4, 6, 7, 8],
            "Gauge": (18, 500),
            "Barrels": (1, 6),
            "Evacuator": ["Y", "N"],
            "Ejector": ["Y", "N"],
            "Rail": ["Y", "N"],
            "RailLimit": (1000, 20000)
        },
        "Engine": {
            "PPM": (200, 800),
            "PPV": (10, 200),
            "PPB": (10, 200)
        }
    },
    "Requirements": {
        "AP": (0, 100),
        "Velocity": {
            "min": 500,
            "max": 2000
        }
    },
    "GoalWeights": {
        "KD": (0, 1),
        "CD": (0, 1),
        "RoF": (0, 1),
        "Uptime": (0, 1),
        "Volume": (0, 1),
        "Cost": (0, 1)
    }
}

In [ ]:
parameters = {
    "Configuration": {
        "Shell": {
            "Modules": {
                "Head": "AP",
                "Fuse": None,
                "Base": None
            }
        },
        "Gun": {
            "Loader": [4],
            "Gauge": {
            "min": 100,
            "max": 250
            },
            "Barrels": 1,
            "Evacuator": False,
            "Ejector": False,
            "Rail": False,
            "RailLimit": 1000
        },
        "Engine": {
            "PPM": 500,
            "PPV": 100,
            "PPB": 100
        }
    },
    "Requirements": {
        "AP": 20,
        "Velocity": {
            "min": 500,
            "max": 2000
        },
        "PenReq": None
    },
    "GoalWeights": {
        "KD": 0.5,
        "CD": 0.5,
        "RoF": 0.5,
        "Uptime": 0.5,
        "Volume": 0.5,
        "Cost": 0.5
    }
}


##Implementation

In [4]:
class Shell:
    def __init__(self, configuration):
        # Shell Configuration
        self.head = configuration["Configuration"]["Shell"]["Modules"]["Head"]

        self.body = configuration["Configuration"]["Shell"]["Modules"]["Body"]

        self.base = configuration["Configuration"]["Shell"]["Modules"]["Base"]
        self.gp = configuration["Configuration"]["Shell"]["Modules"]["GP"]
        self.rc = 0 if not self.rail else configuration["Configuration"]["Shell"]["Modules"]["RC"]

        # Gun Configuration
        self.loader = configuration["Configuration"]["Shell"]["Loader"]
        self.gauge = configuration["Configuration"]["Gun"]["Gauge"]
        self.barrels = configuration["Configuration"]["Gun"]["Barrels"]
        self.evacuator = configuration["Configuration"]["Gun"]["Evacuator"]
        self.ejector = configuration["Configuration"]["Gun"]["Ejector"]
        self.rail = configuration["Configuration"]["Gun"]["Rail"]
        self.rail_limit = configuration["Configuration"]["Gun"]["RailLimit"]
        self.rail_draw_scalar = configuration["Configuration"]["Gun"]["RailDrawScalar"]

        # Engine Configuration
        self.ppm = configuration["Configuration"]["Engine"]["PPM"]
        self.ppv = configuration["Configuration"]["Engine"]["PPV"]
        self.ppb = configuration["Configuration"]["Engine"]["PPB"]
        self.ppc = configuration["Configuration"]["Engine"]["PPC"]

        # Misc

    #   KD

    #   CD

    #   RoF

    def reloadTime(self):
        reload = 1 # TODO: Calc

        return reload if self.loader != "Belt" else reload * 1 # TODO: Calc

    def uptime(self):
        if self.loader != 'Belt': return 1
        else:
            firingCycle = 1 # TODO: Calc
            loadingCycle = 1 # TODO: Calc
            return firingCycle/(loadingCycle + firingCycle)

    #   Volume

    def loaderVolume(self):
        ejectorVolume = 2 if self.ejector else 0
        return (self.loader + 1 + ejectorVolume) if self.loader != "Belt" else 1 # TODO: Calc

    def recoilVolume(self):
        return self.totalRecoil() / (self.reloadTime() * 120)

    def coolerVolume(self):
        if self.gp == 0: return 0
        multiBarrelPenalty = 1 + (self.barrels - 1) * 0.2
        boreEvacuatorBonus = 0
        if self.evacuator:
            boreEvacuatorBonus = 1 # TODO: Calc

        return 1 # TODO: Calc

    def chargerVolume(self):
        return self.railDraw() / self.reloadTime() / 200 # rail draw per shot / secs per shot / rail charge per sec per volume (200)

    def magnetVolume(self):
        return self.railDraw() / 5000 # draw per shot / charge capacity per volume (5000), not sure if I should round up or not

    def engineVolume(self):
        return self.railDraw() / self.reloadTime() / self.ppv # calc is rail draw per shot / secs per shot / engine power per volume

    def volumePerIntake(self):
        return max(
            self.loaderVolume() +
            self.recoilVolume() +
            self.coolerVolume() +
            self.chargerVolume() +
            self.magnetVolume() +
            self.engineVolume(),
            2
        )

    #   Cost

    def loaderCost(self):
        return 1 # TODO: Calc

    def recoilCost(self):
        return self.recoilVolume() * costData["Recoil"];

    def coolerCost(self):
        return self.coolerVolume() * costData["Cooler"];

    def chargerCost(self):
        return self.chargerVolume() * costData['Charger'];

    def magnetCost(self):
        return self.magnetVolume() * costData["Magnet"];

    def engineCost(self):
        return self.engineVolume * self.ppc

    def builldCost(self):
        return (
            self.loaderCost() +
            self.recoilCost() +
            self.coolerCost() +
            self.chargerCost() +
            self.magnetCost() +
            self.engineCost()
        )

    def projectileCost(self):
        return (1 # TODO: Calc, direct material cost of proj
                + self.railDraw / self.ppm) # cost of rail energy per shot

    def costPerIntake(self):
        return self.buildCost + (self.projectileCost / self.reloadTime) * self.duration

    #   Misc
    def railDraw(self):
        return min(self.rail_limit, self.maxDraw()) * self.rail_draw_scalar

    def maxDraw(self):
        return 1 # TODO Calc, shells have max draw based on size/components


    def totalRecoil(self):
        return 1 # TODO Calc


    def loaderLength(self):
        return 1000 if self.loader == "Belt" else self.loader * 1000

    def headLength(self):
        return min(self.gauge, headData[self.head].maxLength or 500)

    def bodyLength(self):
        for body in self.body:

        for fuse in self.fuse:
            len += min(self.gauge, bodyData[fuse].maxLength or 500)
        return len

    def baseLength(self):
        return min(self.gauge, baseData[self.base].maxLength or 500) if self.base else: 0

    def projectileLength(self):
        return self.headLength() + self.bodyLength() + self.baseLength()

    def casingLength():
        return (self.gp + (self.rc or 0)) * self.gauge

    def totalLength(self): # will be used to apply penalty if totalLength > loaderLength
        return self.projectileLength() + self.casingLength()

    def effectiveProjectileModuleCount(self):
        return self.projectileLength() / self.gauge

In [2]:
loaderLengths = ["Belt", 1, 2, 3, 4, 6, 8]

headData = {
    "AP": {
        "VelocityMod": 1.6,
        "KineticDamageMod": 1.0,
        "ArmorPierceMod": 1.65,
        "ChemMod": 1.0,
        "ShortName": "AP",
    },
    "Sabot": {
        "VelocityMod": 1.6,
        "KineticDamageMod": 0.85,
        "ArmorPierceMod": 2.5,
        "ChemMod": 0.25,
        "ShortName": "Sab",
    },
    "Heavy": {
        "VelocityMod": 1.45,
        "KineticDamageMod": 1.65,
        "ArmorPierceMod": 1.0,
        "ChemMod": 1.0,
        "ShortName": "Hvy",
    },
    "HP": {
        "VelocityMod": 1.45,
        "KineticDamageMod": 1.2,
        "ArmorPierceMod": 1.0,
        "ChemMod": 1.0,
        "ShortName": "HP",
    },
    "Chem": {
        "VelocityMod": 1.3,
        "KineticDamageMod": 1.0,
        "ArmorPierceMod": 0.1,
        "ChemMod": 1.0,
        "IsChem": True,
        "ShortName": "Chem",
    },
    "Special": {
        "VelocityMod": 1.45,
        "KineticDamageMod": 0.1,
        "ArmorPierceMod": 0.1,
        "ChemMod": 1.0,
        "IsChem": True,
        "ShortName": "Spc",
    },
}
bodyData = {
    "SolidBody": {
        "VelocityMod": 1.1,
        "KineticDamageMod": 1.0,
        "ArmorPierceMod": 1.0,
        "ChemMod": 1.0,
        "ShortName": "Sol",
    },
    "SabotBody": {
        "VelocityMod": 1.1,
        "KineticDamageMod": 0.8,
        "ArmorPierceMod": 1.4,
        "ChemMod": 0.25,
        "ShortName": "Sab",
    },
    "ChemBody": {
        "VelocityMod": 1.0,
        "KineticDamageMod": 1.0,
        "ArmorPierceMod": 0.1,
        "ChemMod": 1.0,
        "ShortName": "Chem",
    },
    "Fuse": {
        "VelocityMod": 1.0,
        "KineticDamageMod": 1.0,
        "ArmorPierceMod": 1.0,
        "ChemMod": 1.0,
        "MaxLength": 100,
        "ShortName": "Fuse",
    }
}
baseData = {
    "BaseBleeder": {
        "VelocityMod": 1.0,  # special +0.15 bonus handled elsewhere
        "KineticDamageMod": 1.0,
        "ArmorPierceMod": 1.0,
        "ChemMod": 1.0,
        "MaxLength": 100,
        "ShortName": "BB",
    },
    "Tracer": {
        "VelocityMod": 1.0,
        "KineticDamageMod": 1.0,
        "ArmorPierceMod": 1.0,
        "ChemMod": 1.0,
        "MaxLength": 100,
        "ShortName": "Tracer",
    }
}



In [3]:
costData = {
    "InputFeeder": 50,
    "Cooler": 50,
    "RecoilAbsorber": 80, # 80 per meter, longer ones scale linearly
    "RailCharger": 400,
    "RailMagnet": 300,
    "Ejector": 10,
    "Loader": {
        "Belt": 600,
        1: 240,
        2: 300,
        3: 330,
        4: 360,
        5: 390,
        6: 420,
        7: 450,
        8: 480,
    },
    "Clip": {
        "Belt": 160,
        1: 160,
        2: 200,
        3: 220,
        4: 240,
        5: 260,
        6: 280,
        7: 300,
        8: 320,
    },
}